import flats as flats
<h1>Flat prices analysis in St. Petersburg</h1>

<div style="background:#abd5f5; border:1px solid #b3deff; padding: 20px">
    <h2 style="color:#002b63">Table of content</h2>
<ul>
    <li>Indroduction</li>
    <li>Data Preprocessing</li>
    <li>Visualization</li>
    <li>Model building</li>
    <li>Results</li>
    <li>Conclusion</li>
</ul>
    </div>

<h2>Indroduction</h2>

This analysis focuses on flat prices in St. Petersburg.

<h2>Data Preprocessing</h2>

I am going to use my own CSV file. I created it using data from https://spb.cian.ru/kupit-kvartiru/ This file contains a list of St. Petersburg flats for sale.

In [1]:
import pandas as pd
import numpy as np
import re #regexp
from ipywidgets import IntProgress #for progress bar
from IPython.display import display

In [2]:
flats=pd.read_csv('data/flats_all.csv',sep=';')
flats.head()

,link,price,total_area,living_area,kitchen_area,floor,type,height,bathrooms,balconies,...,renovation,view,rooms,city,area,neighborhood,street,metro_name,metro_km,number_of_floors
0,https://spb.cian.ru/sale/flat/234775065/,13143249.0,74.13,26.3,22.0,3,Новостройка,3.3,2.0,1.0,...,NaN,NaN,2,Санкт-Петербург,р-н Приморский,Юнтолово,"Планерная ул., 94",Комендантский проспект,1.98,12
1,https://spb.cian.ru/sale/flat/239273301/,12430600.0,47.81,NaN,NaN,2,Новостройка,NaN,0.0,0.0,...,NaN,NaN,1,Санкт-Петербург,р-н Петроградский,Посадский,"ул. Рентгена, 25",Петроградская,1.04,8
2,https://spb.cian.ru/sale/flat/250966190/,13800000.0,44.10,14.1,19.4,2,Новостройка,NaN,0.0,0.0,...,NaN,NaN,1,Санкт-Петербург,р-н Курортный,мкр. Сестрорецк,"ул. Максима Горького, 2Ас2",Беговая,9.57,5
3,https://spb.cian.ru/sale/flat/249950664/,8730851.0,56.29,29.3,10.9,5,Новостройка,NaN,2.0,2.0,...,NaN,NaN,2,Санкт-Петербург,р-н Приморский,Юнтолово,Нью Тайм жилой комплекс,Комендантский проспект,1.98,13
4,https://spb.cian.ru/sale/flat/250766812/,13650000.0,70.80,NaN,10.0,13,Вторичка,2.8,1.0,1.0,...,Евроремонт,На улицу и двор,2,Санкт-Петербург,р-н Приморский,Комендантский аэродром,"аллея Поликарпова, 6к1",Пионерская,1.04,19


Next, I will describe structure of dataframe.

<table>
    <tr>
        <th>N</th>
        <th>Name</th>
        <th>Description</th>
    </tr>
    <tr>
        <td>1</td>
        <td>link</td>
        <td>Link to the page with the flat on cian.ru</td>
    </tr>
    <tr>
        <td>2</td>       
        <td>price</td>
        <td>Flat price</td>
    </tr>
    <tr>
        <td>3</td>
        <td>total_area</td>
        <td>The total area of the flat in square meters</td>
    </tr>
    <tr>
        <td>4</td>        
        <td>living_area</td>
        <td>All bedrooms area plus living room in square meters</td>
    </tr>
    <tr>
        <td>5</td>
        <td>kitchen_area</td>
        <td>Kitchen area in square meters</td>
    </tr>
    <tr>
        <td>6</td>
        <td>floor</td>
        <td>Floor number where the flat is located</td>
    </tr>
    <tr>
        <td>7</td>
        <td>type</td>
        <td>Building type: new (still under construction) or old</td>
    </tr>
    <tr>
        <td>8</td>
        <td>height</td>
        <td>Ceiling height</td>
    </tr>    
    <tr>
        <td>9</td>
        <td>bathrooms</td>
        <td>Number of bathrooms</td>
    </tr>
    <tr>
        <td>10</td>
        <td>balconies</td>
        <td>Number of balconies</td>
    </tr>
    <tr>
        <td>11</td>
        <td>finishing</td>
        <td>Finishing type of new flats: renovated and without</td>
    </tr>    
    <tr>
        <td>12</td>
        <td>year_of_construction</td>
        <td>Year when the building was or will be built</td>
    </tr>    
    <tr>
        <td>13</td>
        <td>layout</td>
        <td>The layout determines how the entrance to the rooms is located: through another room or through a corridor</td>
    </tr>    
    <tr>
        <td>14</td>
        <td>renovation</td>
        <td>Type of renovation</td>
    </tr>    
    <tr>
        <td>15</td>
        <td>view</td>
        <td>View from windows</td>
    </tr> 
    <tr>
        <td>16</td>
        <td>rooms</td>
        <td>Number of rooms</td>
    </tr>     
    <tr>
        <td>17</td>
        <td>city</td>
        <td>City name</td>
    </tr>     
    <tr>
        <td>18</td>
        <td>area</td>
        <td>Area name</td>
    </tr>     
    <tr>
        <td>19</td>
        <td>neighborhood</td>
        <td>Neighborhood name</td>
    </tr> 
    <tr>
        <td>20</td>
        <td>street</td>
        <td>Street name</td>
    </tr>     
    <tr>
        <td>21</td>
        <td>metro_name</td>
        <td>The closest metro station</td>
    </tr>     
    <tr>
        <td>22</td>
        <td>metro_km</td>
        <td>Distance to the closest metro station in km</td>
    </tr>     
    <tr>
        <td>23</td>
        <td>number_of_floors</td>
        <td>Number of floors in the buildings</td>
    </tr>
</table>

First, I count the price for a square meter.

In [22]:
flats['price_meter']=flats['price']/flats['total_area']

Let's check values in the column 'rooms'.

In [4]:
flats['rooms'].value_counts()

2                 353
3                 333
1                 309
Студия            169
4                 122
5                  52
Многокомнатная     39
Апартаменты        31
Квартира            6
Name: rooms, dtype: int64

Next, I replace 'Студия' with 0.5 and 'Многокомнатная' with 6. Values 'Апартаменты', 'Квартира' don't contain information about the number of rooms so I replace them with the most frequent value. Also, I convert the column into the float format.

In [5]:
flats['rooms']=flats['rooms'].replace(['Студия','Многокомнатная','Многокомнатные'],['0.5',6,6])
flats['rooms']=flats['rooms'].replace(['Апартаменты','Квартира'],flats['rooms'].value_counts().idxmax())
flats['rooms']=flats['rooms'].astype(float)
flats[['rooms']].head()

,rooms
0,2.0
1,1.0
2,1.0
3,2.0
4,2.0


Next, I will translate all categorical values into English.

In [6]:
flats['type']=flats['type'].replace(['Вторичка','Новостройка'],['secondary','new'])
flats['view']=flats['view'].replace(['На улицу и двор','Во двор','На улицу'],['both','view_courtyard','view_street'])
flats['finishing']=flats['finishing'].replace(['Чистовая','Нет','Черновая'],['fine','none','rough'])
flats['layout']=flats['layout'].replace(['Изолированная','Смежно-изолированная','Смежная'],['isolated','mixed','adjoining'])
flats['renovation']=flats['renovation'].replace(['Евроремонт','Без ремонта','Косметический','Дизайнерский'],
                                                ['euro','none','redecorating','designer'])

The next step is finding and removing missing data. Firstly, let's identify NaNs and zeros.

In [7]:
pd.isna(flats).sum()

link                       0
price                      0
total_area                 0
living_area              159
kitchen_area             261
floor                      0
type                       0
height                   685
bathrooms                  0
balconies                  0
finishing               1187
year_of_construction     203
layout                   898
renovation               462
view                     560
rooms                      0
city                       0
area                       0
neighborhood              13
street                    13
metro_name               135
metro_km                   0
number_of_floors           0
dtype: int64

Now I will find out the proportion between living_area, kitchen_area, total_area and replace missing data with the most common proportion.

In [8]:
flats['living_part']=flats['living_area']/flats['total_area']
flats['kitchen_part']=flats['kitchen_area']/flats['total_area']
flats['living_area']=flats['living_area'].combine_first(flats['total_area']*flats['living_part'].mean())
flats['kitchen_area']=flats['kitchen_area'].combine_first(flats['total_area']*flats['kitchen_part'].mean())
flats.drop(['living_part','kitchen_part'],axis='columns',inplace=True)

I replace missing data in columns 'view', 'finishing', 'layout', 'renovation','year_of_construction' with the most popular value.

In [9]:
flats['view']=flats['view'].replace(np.nan,flats['view'].value_counts().idxmax())
flats['finishing']=flats['finishing'].replace(np.nan,flats['finishing'].value_counts().idxmax())
flats['layout']=flats['layout'].replace(np.nan,flats['layout'].value_counts().idxmax())
flats['renovation']=flats['renovation'].replace(np.nan,flats['renovation'].value_counts().idxmax())
flats['year_of_construction']=flats['year_of_construction'].replace(np.nan,flats['year_of_construction'].value_counts().idxmax())

I replace the height with the mean value. 

In [10]:
flats['height']=flats['height'].replace(np.nan,flats['height'].mean())

Finally, I drop lines with missing values in 'street'

In [18]:
flats.drop(flats[flats['street'].isna()].index,axis='rows',inplace=True)

<h2>Visualization</h2>

In [ ]:
import googlemaps
f=open('keys/api_google_maps_key')
mykey=f.read()
f.close()
gmaps = googlemaps.Client(key=mykey)

In [ ]:
flats['lat']=0.0
flats['lng']=0.0

progress = IntProgress(min=0, max=len(flats), value=0) #progress bar
display(progress)

for i in range(len(flats)):
    geocode_result = gmaps.geocode(flats.loc[i,'city']+', '+flats.loc[i,'street'])
    flats.loc[i,'lat']=geocode_result[0]['geometry']['location']['lat']
    flats.loc[i,'lng']=geocode_result[0]['geometry']['location']['lng']
    progress.value = i

In [ ]:
flats.to_csv('flats_cian_1.csv',sep=';',index=False)

In [ ]:
flats=pd.read_csv('flats_cian_1.csv',sep=';')

Next I create a map

In [ ]:
import folium
map_piter = folium.Map(location=[59.9810199, 30.3540484], zoom_start=9)

# add markers to map
for lat, lng, price, name in zip(flats['lat'],flats['lng'],flats['price'],flats['link']):
    label = '{}'.format(round(price))
    label = folium.Popup(label, parse_html=True)
    if price<5000000:
        clr='#00ffff'
    elif price<10000000:
        clr='#91e2da'
    elif price<15000000:
        clr='#d9a694'
    elif price<20000000:
        clr='#eb8473'
    elif price<25000000:
        clr='#f75b53'
    else:
        clr='#ff0035'
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=clr,
        fill=True,
        fill_color=clr,
        fill_opacity=0.8,
        parse_html=False).add_to(map_piter)
    
map_piter

<h1>Model building</h1>

Here I'm going to build the predictive model. Price is dependent value. Other features are independent ones. 

Firstly, I need to prepare the dataframe. I create a copy of 'flats' and delete unnecessary fielda

In [ ]:
flats_model=flats.copy()
flats_model.drop(['link','city','neighborhood','street','lat','lng','type','view','finishing','layout',
                  'renovation','area','metro_name'],axis='columns',inplace=True)

Next I need to turn all categorical values into numerical. I use 'get_dummies' function from pandas.

In [ ]:
cat_df=flats[['type','view','finishing','layout','renovation']]
cat_df=pd.get_dummies(data=cat_df,drop_first=True)

#area_df=pd.get_dummies(flats_model['area'])
#metro_name_df=pd.get_dummies(flats_model['metro_name'])

#Join all the dataframes in one
flats_model=pd.concat([flats_model,cat_df],axis=1)

# Check the result
flats_model.head()

Next step is searching correlation among all features.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

corr=flats_model.corr()
fig, ax = plt.subplots(figsize=(15,10))

heatmap=sns.heatmap(corr,center=0,ax=ax)

In [ ]:
corr[(corr['price']<-0.2)|(corr['price']>0.2)]['price']

In [ ]:
drop_columns=corr[(corr['price']>=-0.2)&(corr['price']<=0.2)].index
flats_model.drop(drop_columns,axis='columns',inplace=True)

In [ ]:
corr=flats_model.iloc[:,1:].corr()
corr[corr.iloc[:]>0.7]

In [ ]:
flats_model.drop(['living_area'],axis='columns',inplace=True)

In [ ]:
flats_model.head()

In [ ]:
flats_model.corr()

In [ ]:
#Построить модели для r=0.1 и для всех параметров
#Построить модели для цены за квадратный метр
#Разделить файла на вебскрапинг и анализ